In [1]:
%load_ext autoreload
%autoreload 2
import sklearn
from sklearn.preprocessing import StandardScaler
import sklearn.svm as svm
from sklearn.neural_network import (MLPClassifier, MLPRegressor)
import numpy as np
from agents.knn import *
from settings import settings

# Congfig cell

In [2]:
from agents.triple_model_agent import extract_state

datasets_paths = [
#     '/home/kku/Documents/engineering-thesis/important_logs',
    '/home/kku/Dropbox/datasets/cg_track_2_2',
#     '/home/kku/Dropbox/CG_track_3'
]


speed_actions_labels = [
    {'accel':0, 'brake': 0},
    {'accel':1, 'brake': 0},
    {'accel':0, 'brake': 1}
]
state_keys = [
    'speedX',
    'speedY',
    'speedZ',
    'angle',
    'trackPos',
#     'distFromStart',
    'track',
    'wheelSpinVel'
]

test_size = 0.3

regressor_def = MLPRegressor(
    (500, 30),
    activation='tanh', max_iter=800, n_iter_no_change=30)

classifier_def = MLPClassifier(
    (200,20),
    activation='tanh',
    max_iter=800
)
classifier_steer_decision_def = MLPClassifier(
    (350,20),
    activation='tanh',
    max_iter=800
)
# regressor_def = svm.SVR()
# classifier_def = svm.SVC(gamma='scale', class_weight='balanced', cache_size=1500)

In [3]:
data = [
    x
    for path in datasets_paths
    for x in parse_dataset(path) 
    if not (x['speedX'] == 0 and x['accel'] == 0)
]
states = np.array(extract_state(data, state_keys, data_is_array=True))

def st_dec(x):
    if x < 0: return -1
    if x > 0: return 1
    return 0
 
steer_decisions = np.array([st_dec(x['steer']) for x in data])
steer_actions = np.array([(x['steer']) for x in data])
speed_actons = np.array([
    speed_actions_labels.index({
        'accel': x['accel'],
        'brake': x['brake']
    }) 
    for x in data
])

In [4]:
MULTPLE_INPUTS = False
if MULTPLE_INPUTS:
    states = np.array([
        [
            *states[i + 1],
            *states[i],
            steer_actions[i],
            speed_actons[i],
        ] 
        for i in range(states.shape[0] - 1)
    ])
    speed_actons = speed_actons[1:]
    steer_actions = steer_actions[1:]

In [5]:
X, Y = sklearn.utils.shuffle(states, steer_actions)
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=test_size)
X_train.shape

(54678, 29)

In [6]:
scaler = StandardScaler()  
scaler.fit(X_train)  
print(scaler.transform(X_train).shape)
regressor = regressor_def
regressor.fit(
    scaler.transform(X_train),
    Y_train
) 

(54678, 29)


MLPRegressor(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(500, 30), learning_rate='constant',
       learning_rate_init=0.001, max_iter=800, momentum=0.9,
       n_iter_no_change=30, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [7]:
print(
    'Train: ',
    regressor.score(
        scaler.transform(X_train),
        Y_train
    ),
    sklearn.metrics.mean_squared_error(
        Y_train,
        regressor.predict(scaler.transform(X_train))
    )
)
print(
    'Test: ',
    regressor.score(
        scaler.transform(X_test),
        Y_test
    ),
    sklearn.metrics.mean_squared_error(
        Y_test,
        regressor.predict(scaler.transform(X_test))
    )
)
(regressor.predict(scaler.transform(X_test)) * Y_test < 0).sum(), 

Train:  0.8173505946002667 0.004378162760488749
Test:  0.7996415032205788 0.004689847896896766


(219,)

In [8]:
nothing_ids = Y_test == 0
diffs = np.abs(
        Y_test[nothing_ids] -
    regressor.predict(scaler.transform(X_test[nothing_ids]))
)
diffs.sort()
(diffs > 0.05).sum(), diffs.shape

(2391, (16015,))

In [9]:
X, Y = sklearn.utils.shuffle(
    scaler.transform(states),
    steer_decisions
)
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=test_size)
classifier_steer_decision = classifier_steer_decision_def
classifier_steer_decision.fit(
    X_train,
    Y_train
)

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(350, 20), learning_rate='constant',
       learning_rate_init=0.001, max_iter=800, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [10]:
print(classifier_steer_decision.score(
    X_train,
    Y_train
))
print(classifier_steer_decision.score(
    X_test,
    Y_test
))
print(sklearn.metrics.confusion_matrix(Y_train, classifier_steer_decision.predict(X_train)))
sklearn.metrics.confusion_matrix(Y_test, classifier_steer_decision.predict(X_test))

0.942627747905922
0.9033882393104037
[[ 4759   737     3]
 [  437 36015   842]
 [    0  1118 10767]]


array([[ 1792,   514,     2],
       [  416, 15001,   589],
       [    0,   743,  4377]])

In [11]:
X, Y = sklearn.utils.shuffle(
    np.hstack((
        scaler.transform(states),
#         (
#             regressor.predict(scaler.transform(states)) * 
#             classifier_steer_decision.predict(scaler.transform(states))
#         )[:, np.newaxis]
        steer_actions[:, np.newaxis]
    )),
    speed_actons
)
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=test_size)
classifier = classifier_def
classifier.fit(
    X_train,
    Y_train
)

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(200, 20), learning_rate='constant',
       learning_rate_init=0.001, max_iter=800, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [15]:
def steer_data_to_pred(X):
    X[:, -1] = regressor.predict(X)
# steer_data_to_pred(X_train),
# steer_data_to_pred(X_test)
    
print(classifier.score(
    X_train,
    Y_train
))
print(classifier.score(
    X_test,
    Y_test
))
print(sklearn.metrics.confusion_matrix(Y_train, classifier.predict(X_train)))
sklearn.metrics.confusion_matrix(Y_test, classifier.predict(X_test))

0.9893192874647939
0.98293078433046
[[11912   277    30]
 [  218 41075     4]
 [   55     0  1107]]


array([[ 4985,   168,    22],
       [  185, 17567,     3],
       [   22,     0,   482]])

In [16]:
from utils import generate_id
import os
def dump(path='saved_model'):
    print(f'Dumping model to: {path}')
    os.system(f'mkdir {path}')
    np.savez(
        f'{path}/parameters.npz',
        **{
            'state_keys': state_keys,
            'speed_actions_labels': speed_actions_labels,
        }
    )
    joblib.dump(scaler, f'{path}/scaler')
    joblib.dump(regressor, f'{path}/steer_regressor')
    joblib.dump(classifier, f'{path}/speed_classifier')
    joblib.dump(classifier_steer_decision, f'{path}/steer_classifier')
dump(f'models/{generate_id()}')

Dumping model to: models/2019-01-08_22:33:11_288
